In [1]:
import copy
import random
from collections import defaultdict
import pandas as pd
from src import SRC_DIR

In [177]:
class Resource:
    def __init__(self, name: str, capacity: int):
        self.name = name
        self.capacity = capacity
        self.available = capacity
        self.free_up_resources = defaultdict(int)



class Activity:
    def __init__(self, name: str, duration: float, resources: dict):
        self.name = name
        self.duration = duration
        self.resources = resources



class Variant:
    def __init__(self, name: str, activities: list):
        self.name = name
        self.activities = activities



class Trace:
    def __init__(self, variant: Variant, entry_time=0):
        self.variant_name = variant.name
        self.activities = variant.activities
        self.entry_time = entry_time

        self.todo_activities = copy.deepcopy(variant.activities)
        self.doing_activities = []  # {<activity>, <start_time>, <end_time>}
        self.done_activities = [] # {<activity>, <start_time>, <end_time>}
        self.start_time = None
        self.end_time = None



class Simulator:
    def __init__(self, traces, resources):
        self.current_time = 0
        self.traces = [Trace(variant, details['Entry_Time'])
                       for variant, details in traces
                       for ind in range(details['Count'])]
        self.resources = {resource.name: resource for resource in resources}


    def update_trace_doing_activities_state(self, trace):
        # Update the state of activities being done in the trace
        for ind, doing_item in enumerate(trace.doing_activities):
            if doing_item['end_time'] == self.current_time:
                done_activity = trace.doing_activities.pop(ind)
                trace.done_activities.append(done_activity)
                if len(trace.todo_activities) == 0 and len(trace.doing_activities) == 0:
                    trace.end_time = self.current_time


    def free_up_resources(self):
        # Free up resources that are no longer in use
        for resource_name, resource in self.resources.items():
            if self.current_time in resource.free_up_resources.keys():
                resource.available += resource.free_up_resources[self.current_time]
                resource.free_up_resources.pop(self.current_time)


    def allocate_resources(self, activity):
        # Check if required resources are available
        for resource_name, resource_required in activity.resources.items():
            resource = self.resources[resource_name]
            if resource.available < resource_required:
                return False
        # Allocate resources
        for resource_name, resource_required in activity.resources.items():
            self.resources[resource_name].available -= resource_required
            self.resources[resource_name].free_up_resources[self.current_time + activity.duration] += resource_required
        return True


    def run(self, duration):
        while self.current_time < duration:
            # Free up resources
            self.free_up_resources()

            for trace in self.traces:
                # Update trace state doing state
                self.update_trace_doing_activities_state(trace)

                # Pass done traces and the traces that their start data is bigger than current time                
                if len(trace.todo_activities) == 0 or len(trace.doing_activities) > 0 or trace.entry_time > self.current_time:
                    continue

                # Select first activity of trace
                activity = trace.todo_activities[0]

                # Allocate resources
                if not self.allocate_resources(activity):
                    continue

                # Assign Trace Start Time
                if len(trace.doing_activities) == 0 and len(trace.done_activities) == 0:
                    trace.start_time = self.current_time

                # Moving activity from todo to doing
                doing_activity = trace.todo_activities.pop(0)
                trace.doing_activities.append({'activity': doing_activity,
                                               'start_time': self.current_time,
                                               'end_time': self.current_time + activity.duration})

            self.current_time += 1


    def report_traces(self):
        trace_info_list = []
        for trace in self.traces:
            sum_of_activity = sum(item.duration for item in trace.activities)    
            duration = None
            waiting_time = None

            if trace.end_time:
                duration = trace.end_time - trace.start_time
                waiting_time = trace.end_time - trace.entry_time - sum_of_activity

            trace_info_list.append([trace.variant_name,
                                    trace.start_time,
                                    trace.end_time,
                                    sum_of_activity,
                                    duration,
                                    waiting_time])

        return pd.DataFrame(trace_info_list, columns=['Variant', 
                                                      'CaseStartTime',
                                                      'CaseEndTime',
                                                      'TotalDurationOfActivity',
                                                      'CaseTotalDuration',
                                                      'WaitingTime'])


    def create_eventlog(self):
        eventlog = []
        for case_ind, trace in enumerate(self.traces):
            for activity in trace.done_activities:
                eventlog.append([f"Case_{case_ind+1}",
                                 trace.variant_name,
                                 trace.start_time,
                                 trace.end_time,
                                 activity['activity'].name,
                                 activity['start_time'],
                                 activity['end_time']])

        return pd.DataFrame(eventlog, columns=['Case',
                                               'Variant', 
                                               'CaseStartTime',
                                               'CaseEndTime',
                                               'ActivityName',
                                               'ActivityStartTime',
                                               'ActivityEndTime'])

In [178]:
# Define resources
resource1 = Resource("Resource 1", 3)
resource2 = Resource("Resource 2", 2)
resource3 = Resource("Resource 3", 2)
resources = {resource1, resource2, resource3}


# Define activities
activity1 = Activity("Activity 1", 5, {"Resource 1": 1})
activity2 = Activity("Activity 2", 9, {"Resource 1": 1, "Resource 2": 2})
activity3 = Activity("Activity 3", 7, {"Resource 2": 1})
activity4 = Activity("Activity 4", 3, {"Resource 3": 1})


# Define Variants
variant1 = Variant("Variant 1", [activity1, activity2, activity3])
variant2 = Variant("Variant 2", [activity3, activity2, activity4])
variant3 = Variant("Variant 3", [activity4, activity1, activity3, activity2, activity1])


# Define Traces
traces = [(variant1, {'Count': 10, 'Entry_Time': 0}),
          (variant1, {'Count': 100, 'Entry_Time': 100}),
          (variant2, {'Count': 20, 'Entry_Time': 10}),
          (variant3, {'Count': 15, 'Entry_Time': 5}),
         ]


# Create and run the simulator
simulator = Simulator(traces, resources)
simulator.run(duration=1000)

In [179]:
simulator.report_traces()

,Variant,CaseStartTime,CaseEndTime,TotalDurationOfActivity,CaseTotalDuration,WaitingTime
0,Variant 1,0,21.0,21,21.0,0.0
1,Variant 1,0,51.0,21,51.0,30.0
2,Variant 1,0,67.0,21,67.0,46.0
3,Variant 1,5,83.0,21,78.0,62.0
4,Variant 1,5,99.0,21,94.0,78.0
...,...,...,...,...,...,...
140,Variant 3,20,NaN,29,NaN,NaN
141,Variant 3,20,NaN,29,NaN,NaN
142,Variant 3,23,NaN,29,NaN,NaN
143,Variant 3,23,NaN,29,NaN,NaN


In [180]:
event_log = simulator.create_eventlog()
event_log

,Case,Variant,CaseStartTime,CaseEndTime,ActivityName,ActivityStartTime,ActivityEndTime
0,Case_1,Variant 1,0,21.0,Activity 1,0,5
1,Case_1,Variant 1,0,21.0,Activity 2,5,14
2,Case_1,Variant 1,0,21.0,Activity 3,14,21
3,Case_2,Variant 1,0,51.0,Activity 1,0,5
4,Case_2,Variant 1,0,51.0,Activity 2,35,44
...,...,...,...,...,...,...,...
288,Case_144,Variant 3,23,NaN,Activity 1,44,49
289,Case_144,Variant 3,23,NaN,Activity 3,392,399
290,Case_145,Variant 3,26,NaN,Activity 4,26,29
291,Case_145,Variant 3,26,NaN,Activity 1,44,49


In [6]:
a = [[1, 5], 2]
b = a

In [7]:
a

[[1, 5], 2]

In [8]:
b

[[1, 5], 2]

In [9]:
b[0].append(9)

In [10]:
list: mutable
tuple: immutable
dictionay: mutable
set: mutable
sting: immutable

pd.DataFrame(): mutable


SyntaxError: illegal target for annotation (685239748.py, line 7)

In [ ]:
class MyClass:
    def __init__(self):
        pass


    def __len__(self):
        print('AAAAAAA')
        return 1


    def __enter__():


    def __exit__():


    def __iter__():
        

In [ ]:
instance1 = MyClass()

In [ ]:
len(instance1)

In [ ]:
with pd.read_csv() as a:
    

In [ ]:
a[0]

In [ ]:
class Student:
    def __init__(self, name, fname, national_code, age, gender, city):
        self.name = name
        self.fname = fname
        self.national_code = national_code
        self.age = age
        self.gender = gender
        self.city = city


    def calculate_grade(self):
        print('Salam')


    def say_hello():
        print('Salam')

In [ ]:
student_1 = Student(name='Kourosh',
                    fname='Hasani',
                    national_code=615646561616,
                    age=25,
                    gender='male',
                    city='Tehran'
                    )

In [ ]:
student_1.fname

In [ ]:
def calculate_sum(a, b, c):
    print(a)
    print(b)
    print(c)

In [ ]:
lst = [1, 2, 3]

In [ ]:
calculate_sum(*lst)

In [ ]:
df.groupby('case').agg(
    
)

In [ ]:
student_1.calculate_grade()    ~     Student.calculate_grade(student_1)

In [ ]:
Student.say_hello()

In [ ]:
def func(a):
    pass

In [ ]:
func()

In [ ]:
student_1   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_2   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_3   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_4   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
student_5   ->   (name, fmaily_name, national_code, age, gender, city, grade, ...)
...

In [ ]:
class Student:
    def __init__(self, name, family_name, national_code, age):
        self.name = name
        self.family_name = family_name
        self.national_code = national_code
        self.age = age
        
        
    def say_hello(self):
        print('Hellooo')
        
        
    def func():
        pass

In [ ]:
student = Student(name='Kourosh', family_name='Hasani', national_code="00000000000", age=30)

In [ ]:
student.national_code

In [ ]:
student.say_hello() #     ~      Student.say_hello(student)

In [ ]:
Student.func()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame([])
df1 = pd.DataFrame([])
df2 = pd.DataFrame([])

In [ ]:
class Student:
    def __init__(self, name, last_name, national_code, age, gender):
        self.name = name
        self.last_name = last_name
        self.national_code = national_code
        self.age = age
        self.gender = gender
        self.course = {}


    def set_grade(self, course_name, grade):
        self.course[course_name] = grade
        print(f"Course: {course_name}    -   Grade: {grade}")
        Student.graduate()


    def __str__(self):
        return f"Name: {self.name}  -  Last Name: {self.last_name}  -  National Code: {self.national_code}"


    def __call__(self):
        print(f"Name: {self.name}  -  Last Name: {self.last_name}  -  National Code: {self.national_code}")


    def __truediv__(self, x):
        return self.name + '_' + x.name

In [121]:
class Student:
    def __init__(self, name, last_name):
        self.name = name
        self.last_name = last_name


    def say_hello():
        print('Hello')


    def greeting(self):
        Student.say_hello()
        print(f"My name is {self.name} {self.last_name}")


    def __str__(self):
        return f"My name is {self.name} {self.last_name}"


    def __len__(self):
        return len(self.name)


    def __call__(self):
        print(f"My name is {self.name} {self.last_name}")


    def __getitem__(self, x):
        return self.name[x]

In [122]:
student1 = Student('Kourosh', 'Hasani')
student2 = Student('Ali', 'Ziaei')

student1[2]

'u'

In [123]:
id(student1)

139793333093280

In [125]:
id(student2)

139793333092208

In [105]:
len(student1)

19

In [106]:
student1()

My name is Kouroshasdfasdfasdf Hasani


In [209]:
a = (1, 2, 3)
b = a

In [210]:
b.append(4)

AttributeError: 'tuple' object has no attribute 'append'

In [212]:
hash([1, 2])

TypeError: unhashable type: 'list'

In [213]:
2 + 3

5

In [230]:
lst = [str(item) for item in range(10000000)]

In [235]:
my_set = {str(item): item for item in range(10000000)}

In [233]:
%%timeit
"5564484" in lst

61.7 ms ± 920 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [236]:
%%timeit
"5564484" in my_set

25.6 ns ± 0.969 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
mutable   ->   unhashable

In [243]:
{
    {5, 6}: 5
}

TypeError: unhashable type: 'set'

In [239]:
string = "korosh"
string[2] = 3

TypeError: 'str' object does not support item assignment

In [244]:
myset = {1, 2, 3}

In [245]:
myset.add(4)

In [246]:
myset

{1, 2, 3, 4}

In [1]:
a = "asdfjaslkdfjlasjdf;asdklf;alskdjf;asdl"

In [2]:
hash(a)

-2827926599895846315

In [3]:
-8790531718727714585

529344067295497451

In [7]:
a = [1, 2, 3, 4]

In [8]:
import hashlib

my_string = "My String!"

my_string_bits  = my_string.encode('utf-8')

secret_thing = hashlib.sha256(my_string_bits)

secret_thing.hexdigest()

'ebe94ffa73de030b652b2ba08359b6508042e8994628642acb80c46035f12f62'

In [6]:
dict_[a]

5